In [41]:
import sys
sys.path.append('..')

import model
import agents

import pandas as pd
import numpy as np
from scipy import stats

import os
import sys
import time
import csv
import datetime
import math

import matplotlib.pyplot as plt
import seaborn as sns
from ptitprince import PtitPrince as pt


from IPython.html.widgets import interact, interactive, fixed
from IPython.html import widgets
'''
/anaconda3/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: 
The `IPython.html` package has been deprecated since IPython 4.0. 
You should import from `notebook` instead. 
`IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
'''

import random as rd
import math
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_EVEN


In [9]:
class Memory:
    
    def __init__(self, filename, agent_name, n_trial = 300, n_pattern = 3):
        self.options = []
        self.selection_rate = []
        self.selected_options = []
        self.if_correct = []
        self.value_table = []
        self.n_trial = n_trial
        self.n_pattern = n_pattern
        self.filename = filename
        self.agent_name = agent_name
        self.value_table_flatten = []
        self.weight_table = []
        return
    
    def get_value_table(self):
        return self.value_table
    
    
    def i_trial(self):
        return len(options)
    
    def record(self, varname, value):
        exec('self.'+ varname + '.append(' + str(value)+')')
        return
    
    def save_options(self, value):
        self.options.append(value)
        return
    def save_selected_options(self, value):
        self.selected_options.append(value)
        return
    def save_if_correct(self, value):
        self.if_correct.append(value)
        return
    def save_value_table(self, value):
        #print(value)
        value = np.array(value).copy()
        self.value_table.append(value)
        self.value_table_flatten.append(value.flatten())
        #print(self.value_table[len(self.value_table)-1])
        return
    def save_selection_rate(self, value):
        self.selection_rate.append(value)
        return
    def save_weight_table(self, value):
        self.weight_table.append(value)
        return
    
    def save_data(self, init_val = []):  
        idx2 = ['options', 'selected_options', 'filename', 'agent_name', 'today']
        today = datetime.datetime.today()
        with open('data_table.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow([today, self.n_trial, self.n_pattern, self.filename, self.agent_name, init_val])
        pd.DataFrame([options, selected_options, if_correct, value_table], index =idx2).transpose().to_csv('log/log_' + filename)
        return
        
    def clear_init_data(self):
        idx = ['time_stamp', 'n_trial', 'n_pattern', 'filename', 'agent_name', 'init_val']
        with open('data_table.csv', 'w') as f:
            writer = csv.writer(f)
            writer.writerow(idx)

In [57]:
class Game:
    
    def n_options(self): #used for making int options
        return 2**self.n_pattern
    
    def __init__(self, filename, n_trial = 300, n_pattern = 3, n_option = 3, competitive_rate = .85):
        self.i_trial = 1 #今、何トライアル目なのか。
        self.filename = filename
        self.rate_table = pd.read_csv(filename, header=None).transpose()
        
        self.n_trial = n_trial
        self.n_pattern = n_pattern
        self.n_option = n_option
        self.competitive_rate = competitive_rate
        return
    
    def compare(self, a,b):
        # both a, b should be np.array
        diff = a-b
        if ((diff >= [0,0,0]).all()):
            return "left_is_bigger"
        if ((diff <= [0,0,0]).all()):
            return "right_is_bigger"
        return "these_are_comparable"
    def if_competitive(self, a,b,c):
        ret = [1,1,1]
        aa = np.array(a)
        bb = np.array(b)
        cc = np.array(c)
        ab = self.compare(aa,bb)
        bc = self.compare(bb,cc)
        ca = self.compare(cc,aa)
        if (ab=="right_is_bigger" or ca=="left_is_bigger"):
            ret[0] = 0
        if (bc=="right_is_bigger" or ab=="left_is_bigger"):
            ret[1] = 0
        if (ca=="right_is_bigger" or bc=="left_is_bigger"):
            ret[2] = 0
        return ret
 
    def get_option_arr3(self):
        table = self.get_option_arr0()
        if(self.competitive_rate == -1):
            return table
        r = rd.random()
        if(r <= self.competitive_rate): # if true, it means it should be competitive
            while (sum(self.if_competitive(table[0],table[1],table[2])) == 1): #if not competitive, thus needed to re-generate
                # self.if_competitive(args[])は、選ばれる可能性のあるものを1とする配列を返す
                # 右が選ばれる場合には [1,0,0]が返る。そのため、sum()を取ると1になる。
                # このwhile内では、sum()==1、つまり、competitiveの場合に再度optionを生成する
                table = self.get_option_arr0()
        else: # false condition, it means it should be non-competitive
            while (sum(self.if_competitive(table[0],table[1],table[2])) != 1): # if competitive, thus needed to re-generate
                table = self.get_option_arr0()
        return table
    
    def get_option_arr0(self):
        n_pattern = self.n_pattern
        left = self.get_option_arr1()
        middle = self.get_option_arr1()
        right = self.get_option_arr1()
        while((np.array(left) == middle).all()):
            middle = self.get_option_arr1()
        while((np.array(left) == right).all() and (np.array(middle) == right).all()):
            right = self.get_option_arr1()
        return [left, middle, right]
    def get_option_arr1(self): #配列[0,0,1]などを返す
        ret = []
        for i in range(self.n_pattern):
            ret.append(rd.randint(0,1))
        return ret
    
    def get_option_int3(self):
        arr = self.get_option_int3()
        return [self.arr2int(arr[0]), self.arr2int(arr[1]), self.arr2int(arr[2])]
    def arr2int(self, arr):
        return arr[0]*4+arr[1]*2+arr[2]
    
    def read_csv(self, filename):  # (i, trials, walkSize)
        # filename = 'rate/rateTable_size'+ str(walkSize) + '_' + str(trials) + 'trials_' + str(i).zfill(3) + '.csv'
        a = pd.read_csv(filename, header=None)
        a = a.transpose()
        a.columns =  ['a','b','c']
        self.rate_table = a
        return
    
    #何トライアル目で、何を選んだかで、当たり・ハズレを返す。
    def correct_or_not(self, option_selected, if_int = False):
        if if_int:
            option_selected = self.int2option(option_selected)
        i = self.i_trial
        r = rd.random() * 100.0
        arr = self.rate_table[i-1 : i]
        rate = arr.dot(option_selected)
        rate = rate[self.i_trial - 1] #選択肢から算出される当たり確率 
        self.i_trial += 1 #このトライアルの終了を意味する。
        return (r < rate)

    


In [11]:
class GameFromLog:
    def __init__(self, filename):
        self.filename = filename
        self.read_data()
        self.i_trial = 1 #今、何トライアル目なのか。
        self.rate_table = pd.read_csv(filename, header=None).transpose()
        
        self.n_trial = n_trial
        self.n_pattern = n_pattern
        self.n_option = n_option
        self.competitive_rate = competitive_rate
        return
    def read_csv(self):
        a = pd.read_csv(self.filename, header=None)
        a = a.transpose()
        a.columns =  ['option_left','option_middle','option_right', 'prob_left','prob_middle','prob_right','choice','hit_or_blow','sum_score', 'rt']
        return a
    def read_logfile(self):
        a = self.read_csv(self.filename)
        #print(a.describe())
        a['option_left_str'] = a.option_left
        a['option_middle_str']  = a.option_middle
        a['option_right_str']  = a.option_right
        a.option_left = str2option(a.option_left)
        a.option_middle = str2option(a.option_middle)
        a.option_right = str2option(a.option_right)
        a.prob_left = a.prob_left.astype(float)/100.0
        a.prob_middle = a.prob_middle.astype(float)/100.0
        a.prob_right = a.prob_right.astype(float)/100.0
        a['best_option'] = get_best_exp_option(a.prob_left, a.prob_middle, a.prob_right)
        a.hit_or_blow = str2bool(a.hit_or_blow)
        a.sum_score = a.sum_score.astype(int)
        a.rt = a.rt.astype(float)
        self.log = a
        return a


In [12]:
class Agent:
    def __init__(self, alpha = .1,  beta = .8, value_length=8, init_value = .7):
        # params
        self.alpha = alpha
        self.beta = beta
        self.value_table = np.ones(value_length)*init_value
        self.init_value_table = self.value_table
        return
    def op2int(self, op):
        return op[0]+op[1]*2+op[2]*4
    def update(self, option, value): # td learning
        before = self.value_table[option]
        self.value_table[option] = before + self.alpha * ( value*self.value_range - before)
        return self.value_table
    
    def get_values(self, ops): # get values of options now shown
        ret = []
        for op in ops:
            ret.append(self.get_value(op))
        return np.array(ret)
    def get_value(self, op):
        idx = op2int(op)
        return self.value_table[idx]
    
    def get_choice_rate(self, op): # get select_rate from values of options now shown
        val= self.get_values(op)
        ex = np.exp(self.beta * val)
        below = sum(ex)/100.0
        return ex/below
    
    def get_choice(self, options): # get random float. using select_rate, get_choice
        r = rd.random()*100.0
        rates = self.get_choice_rate(options)
        for rate, option in zip(rates, options):
            if(r < rate):
                return option
            r -= rate
        return options[-1]
                
    def get_init_val(self):
        return list('alpha', self.alpha, 'beta', self.beta, 'value_table', str(self.init_value_table))
    

In [13]:
class AgentSimpleRL(Agent):
    agent_name = "simpleRL"
    value_range = 1
    option_type = 'int'
    legend = ['000', '001', '010', '011', '100', '101', '110', '111']
    
    def __init__(self, alpha = .1,  beta = .8, value_length=8, init_value = .7):
        super(Agent).__init__(alpha=alpha, beta=beta, value_length=value_length, init_value=init_value)
    
    def update(self, option, value): # td learning
        before = self.value_table[option]
        self.value_table[option] = before + self.alpha * ( value*self.value_range - before)
        return self.value_table

In [14]:
class AgentDimensionRL0L(Agent):
    agent_name = "dimensionRL0"
    value_range = 1 #value_table
    option_type = 'array'
    legend = ['0', '1', '2']
    
    def __init__(self, alpha = .1,  beta = .8, value_length=3, init_value = .7):
        super(Agent).__init__(alpha=alpha, beta=beta, value_length=value_length, init_value=init_value)
        self.value_table_flatten = self.value_table.flatten()
        self.init_value_table = self.value_table
        return
    
    def update(self, option, value):
        for i in range(len(option)):
            if(option[i]):
                self.update2(i, value)
        self.value_table_flatten = self.value_table.flatten()
        return self.value_table
    
    def update２(self, option, value): #option はfeatureの形式に
        before = self.value_table[option]
        self.value_table[option] = before + self.alpha * ( value*self.value_range - before)
        #print(self.value_table)
        return self.value_table
    
    def get_values(self, op):
        if (len(op) == 2):
            return self.get_values2(op)
        if (len(op) == 3):
            return self.get_values3(op)
    
    def get_values2(self, op): #op1: option1
        r1 = np.array(self.value_table).dot(op[0])
        r2 = np.array(self.value_table).dot(op[1])
        return [r1,r2]
    def get_values3(self, op): #op1: option1
        r1 = np.array(self.value_table).dot(op[0])
        r2 = np.array(self.value_table).dot(op[1])
        r3 = np.array(self.value_table).dot(op[2])
        return [r1,r2,r3]
    
    
    def get_select_rate2(self, op): #op1: option1
        [r1, r2] = self.get_values(op)
        e1 = math.exp(self.beta * r1)
        e2 = math.exp(self.beta * r2)
        #print("r1+r2 = " + str([r1, r2]))
        #print("e1+e2 = " + str(e1 + e2))
        return [e1/(e1+e2)*100, e2/(e1+e2)*100]
    
    def get_select_rate3(self, op): #op1: option1
        r1 = self.value_table[op[0]]
        r2 = self.value_table[op[1]]
        r3 = self.value_table[op[2]]
        e1 = math.exp(self.beta * r1)
        e2 = math.exp(self.beta * r2)
        e3 = math.exp(self.beta * r3)
        below = (e1+e2+e3)/100.0  # return persentage
        return [e1/below, e2/below, e3/below]
    
    def get_selection(self, options):
        r = rd.random()*100.0
        if (len(options) == 2):
            rate = self.get_select_rate2(options)
            if(r < rate[0]):
                return options[0]
            else:
                return options[1]
        if(len(options) == 3):
            rate = self.get_select_rate3(options)
            if(r < rate[0]):
                return options[0]
            else:
                if(r <= rate[0] + rate[1]):
                    return options[1]
                else:
                    return options[2]

In [31]:
class AgentGreedyRandom: 
    # optionの順序で単独優位の場合は、それを選ぶ。
    # optionの順序がconflictしている場合は、劣位を除いた中からランダムに選ぶ
    # 点数のベンチマークにするための
    #
    agent_name = "floor"
    value_range = 1 #value_table
    option_type = 'array'
    alpha = -1
    beta = -1
    init_value_table = "none"
    legend = [""]
    
    def __init__(self, alpha=0,beta=0):
        return
    
    def get_values(self, op):
        return -1
    def update(self, option, value):
        return -1
    def get_select_rate3(self, ops): #op1: option1
        idx = self.if_competitive(ops[0],ops[1],ops[2])
        prob = 100.0/sum(idx)
        return np.array(idx)*prob
    def compare(self, a,b):
        # both a, b should be np.array
        diff = a-b
        if ((diff >= [0,0,0]).all()):
            return "left_is_bigger"
        if ((diff <= [0,0,0]).all()):
            return "right_is_bigger"
        return "these_are_comparable"
    
    def if_competitive(self, a,b,c): 
        #順序列で最大の要素の項で1となる配列を返す。
        #左が最大ならば[1,0,0]。中央と右が同等で最大ならば[0,1,1]
        ret = [1,1,1]
        aa = np.array(a)
        bb = np.array(b)
        cc = np.array(c)
        ab = self.compare(aa,bb)
        bc = self.compare(bb,cc)
        ca = self.compare(cc,aa)
        if (ab=="right_is_bigger" or ca=="left_is_bigger"):
            ret[0] = 0
        if (bc=="right_is_bigger" or ab=="left_is_bigger"):
            ret[1] = 0
        if (ca=="right_is_bigger" or bc=="left_is_bigger"):
            ret[2] = 0
        return ret
    
    def get_selection(self, ops):
        r = rd.random()*100.0
        probs = self.get_select_rate3(ops)
        for prob, option in zip(probs, ops):
            if(r < prob):
                return option
            r -= prob
        return options[-1]
                
    def get_init_val(self):
        return list('alpha', self.alpha, 'beta', self.beta, 'value_table', str(self.init_value_table))

In [16]:
class AgentFeatureRL: 
    # 2x3=6 value_table
    #
    agent_name = "featureRL"
    value_range = 1 #value_table
    option_type = 'array'
    legend = ['00', '01', '10', '11', '20', '21']
    
    
    def __init__(self, alpha = .2,  beta = .8, value_length=[3,2]):
        self.alpha = alpha
        self.beta = beta
        self.value_table = np.ones(value_length)*0.7
        self.value_table_flatten = self.value_table.flatten()
        init_v_table = self.value_table
        return
    
    def update(self, option, value):
        for i in range(len(option)):
            self.update3(i, option[i], value)
        self.value_table_flatten = self.value_table.flatten()
        return self.value_table
    
    def update3(self, p,q, value): #option はfeatureRL
        before = self.value_table[p,q]
        self.value_table[p,q] = before + self.alpha * ( value*self.value_range - before)
        #print(self.value_table)
        return self.value_table
    
    def get_values(self, op):
        if (len(op) == 2):
            return self.get_values2(op)
        if (len(op) == 3):
            return self.get_values3(op)
    
    def get_values2(self, op): #op1: option1
        t = self.table2arr()
        r1 = t[0][op[0][0]] + t[1][op[0][1]] + t[2][op[0][2]]
        r2 = t[0][op[1][0]] + t[1][op[1][1]] + t[2][op[1][2]]
        return [r1,r2]
    def table2arr(self):
        t = np.array(self.value_table)
        for i in range(3):
            t[i] = self.sig(t[i])
        return t
        
    def sig(self, arr, beta=1):
        arr = np.array(arr)
        a = math.e ** (arr * beta)
        return a/sum(a)
        
    
    def get_select_rate2(self, op): #op1: option1
        [r1, r2] = self.get_values(op)
        return self.sig([r1,r2], self.beta)*100
    
    def get_select_rate3(self, op): #op1: option1
        r1 = self.value_table[op[0]]
        r2 = self.value_table[op[1]]
        r3 = self.value_table[op[2]]
        e1 = math.exp(self.beta * r1)
        e2 = math.exp(self.beta * r2)
        e3 = math.exp(self.beta * r3)
        below = (e1+e2+e3)/100.0  # return persentage
        return [e1/below, e2/below, e3/below]
    
    def get_selection(self, options):
        r = rd.random()*100.0
        if (len(options) == 2):
            rate = self.get_select_rate2(options)
            if(r < rate[0]):
                return options[0]
            else:
                return options[1]
        if(len(options) == 3):
            rate = self.get_select_rate3(options)
            if(r < rate[0]):
                return options[0]
            else:
                if(r <= rate[0] + rate[1]):
                    return options[1]
                else:
                    return options[2]
                
    def get_init_val(self):
        return list('alpha', self.alpha, 'beta', self.beta, 'value_table', str(self.init_value_table))

In [17]:
class AgentFeatureWeightRL: 
    # 2x3=6 value_table
    #
    agent_name = "featureWeightRL"
    value_range = 1 #value_table
    weight_range = 1
    option_type = 'array'
    legend = ['00', '01', '10', '11', '20', '21']
    
    
    def __init__(self, alpha = .2,  beta = .8, value_length=[3,2]):
        self.alpha = alpha
        self.beta = beta
        self.value_table = np.ones(value_length)*0.5
        self.value_table_flatten = self.value_table.flatten()
        self.weight_table = np.ones(3) #追加
        init_v_table = self.value_table
        return
    
    def update(self, option, value):
        for i in range(len(option)):
            self.update3(i, option[i], value)
        self.value_table_flatten = self.value_table.flatten()
        self.updateWeight()
        return self.value_table
    
    def update3(self, p,q, value): #option はfeatureRL
        before = self.value_table[p,q]
        self.value_table[p,q] = before + self.alpha * ( value*self.value_range - before)
        #print(self.value_table)
        return self.value_table
    
    def updateWeight(self, op_selected, op_other):
        for i in range(3):
            if (op_selected[i] != op_other[i]):
                before = self.weight_table[i]
                self.weight_table[i] = before + self.alpha * ( value*self.weight_range - before)
        return
    
    def get_values(self, op):
        if (len(op) == 2):
            return self.get_values2(op)
        if (len(op) == 3):
            return self.get_values3(op)
    
    def get_values2(self, op): #op1: option1
        t = self.table2arr()
        r1 = t[0][op[0][0]] + t[1][op[0][1]] + t[2][op[0][2]]
        r2 = t[0][op[1][0]] + t[1][op[1][1]] + t[2][op[1][2]]
        return [r1,r2]
    def table2arr(self):
        t = np.array(self.value_table)
        for i in range(3):
            t[i] = self.sig(t[i])
        return t
        
    def sig(self, arr, beta=1):
        arr = np.array(arr)
        a = math.e ** (arr * beta)
        return a/sum(a)
        
    
    def get_select_rate2(self, op): #op1: option1
        [r1, r2] = self.get_values(op)
        return self.sig([r1,r2], self.beta)*100
    
    def get_select_rate3(self, op): #op1: option1
        r1 = self.value_table[op[0]]
        r2 = self.value_table[op[1]]
        r3 = self.value_table[op[2]]
        e1 = math.exp(self.beta * r1)
        e2 = math.exp(self.beta * r2)
        e3 = math.exp(self.beta * r3)
        below = (e1+e2+e3)/100.0  # return rate to persentage
        return [e1/below, e2/below, e3/below]
    
    def get_selection(self, options):
        r = rd.random()*100.0
        if (len(options) == 2):
            rate = self.get_select_rate2(options)
            if(r < rate[0]):
                return options[0]
            else:
                return options[1]
        if(len(options) == 3):
            rate = self.get_select_rate3(options)
            if(r < rate[0]):
                return options[0]
            else:
                if(r <= rate[0] + rate[1]):
                    return options[1]
                else:
                    return options[2]
                
    def get_init_val(self):
        return list('alpha', self.alpha, 'beta', self.beta, 'value_table', str(self.init_value_table))

In [11]:
# テスト
filename = 'rate_table/' + 'rateTable_size20_300trials_001.csv'

m = Memory(filename, 'agent')
m.clear_init_data()

In [18]:
# plot rate 推移

def plot(filename):
    tmp = pd.read_csv(filename, header=None).transpose()
    arr = pd.DataFrame(tmp, columns=['a','b','c'])
    fig, ax = plt.subplots()
    ax = sns.lineplot(data=tmp)
    ax.set_ylim(0,100)
    ax.set_xlim(-1,301)
    ax.axhspan(65,100 ,alpha=0.1, color='red')
    ax.axhspan(55,65 ,alpha=0.05, color='red')
    ax.axvline(100, color='gray')
    ax.axvline(200, color='gray')
    return

In [44]:

filename = '../code/goodTable/rateTable_size20_300trials_0004.csv' # 
g = Game(filename)

a = AgentGreedyRandom()
m = Memory(g.filename, a.agent_name)

print(g.n_trial)
for i in range(g.n_trial):
    #options = g.option2int(g.get_options2())
    options = g.get_options2arr()
    values = a.get_values(options)
    select_rate = a.get_select_rate3(options)
    selection = a.get_selection(options)
    correct_or_not = g.correct_or_not(selection,if_int=False)
    updated_v_table = a.update(selection, correct_or_not)
    
    
    
    m.save_options(options)
    m.save_selected_options(selection)
    m.save_if_correct(correct_or_not)
    m.save_value_table(updated_v_table)
    m.save_selection_rate(select_rate)

print(sum(m.if_correct))
plt.plot(m.if_correct)
plt.figure()
plt.plot(m.value_table_flatten)
#m.save_data(a.get_init_val())


300


AttributeError: 'Game' object has no attribute 'get_options2arr'

In [58]:
#これが本番！!!


def tryAgent(filename, agent_name ='AgentFeatureRL' , without_plot=False, alpha = .05, beta = 10, ):
    
    
    g = Game(filename)
    a = eval(agent_name)(alpha=alpha, beta = beta)
    m = Memory(g.filename, a.agent_name)
    
    for i in range(g.n_trial):
        if (a.option_type == 'array'):
            p=0
            #options = g.get_option_arr3()
        if (a.option_type == 'int'):
            p=0
            #options = g.get_option_int3()
        options = g.get_option_arr3()
        #options = g.get_options()
        #values = a.get_values(options)
        #select_rate = a.get_select_rate3(options)
        selection = a.get_selection(options)
        correct_or_not = g.correct_or_not(selection,if_int=(a.option_type=='arr'))
        #updated_v_table = a.update(selection, correct_or_not)
    
        #m.save_options(options)
        m.save_selected_options(g.arr2int(selection))
        m.save_if_correct(correct_or_not)
        #m.save_value_table(updated_v_table)
        #m.save_selection_rate(select_rate)
    #sns.distplot(m.selected_options)
    if (without_plot == True):
        return sum(m.if_correct)
    ax = plt.figure()
    plt.plot(m.value_table_flatten)
    ax.legend(a.legend)
    #m.save_data(a.get_init_val())
    return sum(m.if_correct)

def average_score_of_agent(runCount = 200, agent_name ='AgentFeatureRL' , 
                           without_plot=True, alpha = .05, beta = 10, filename= 'rateTable_size20_300trials_0004' ):
    filename = 'rate_table/'+ filename + '.csv'
    print(filename)
    a = []
    for i in range(runCount):
        a.append(tryAgent(filename=filename, agent_name =agent_name , without_plot=without_plot, alpha = alpha, beta = beta))
    print(a)
    print(np.array(a).mean())
    plt.figure()
    sns.distplot(a,hist=False)
    plt.figure()
    sns.distplot(a, rug=True, rug_kws={"color": "g"}, 
                             kde_kws={"color": "k", "lw": 3, "label": "KDE"},
                             hist_kws={"histtype": "step", "linewidth": 3,"alpha": 1, "color": "g"})
    return

interact(average_score_of_agent, 
         runCount = widgets.IntSlider(min=1, max=1000, step=10, value=51),
         agent_name = widgets.RadioButtons(options=['AgentSimpleRL', 'AgentFeatureRL0', 'AgentFeatureRL', 'AgentGreedyRandom' ], value='AgentGreedyRandom'),
         filename = widgets.RadioButtons(options=['stable70_25_5', 
                                                  'rateTable_size20_300trials_0001', 
                                                  'rateTable_size20_300trials_0002', 
                                                  'rateTable_size20_300trials_0003',
                                                  'rateTable_size20_300trials_0004'], value='rateTable_size20_300trials_0004'),
         beta = widgets.FloatSlider(min=0, max=20, step=.5, value=5),
         alpha = widgets.FloatSlider(min=0, max=1.0, step=.05, value=.3))

interactive(children=(IntSlider(value=51, description='runCount', max=1000, min=1, step=10), RadioButtons(desc…

<function __main__.average_score_of_agent(runCount=200, agent_name='AgentFeatureRL', without_plot=True, alpha=0.05, beta=10, filename='rateTable_size20_300trials_0004')>

In [125]:
# 直近10回の平均をとる場合
a = np.array(m.if_correct)
b = [1,2,3,4]
print(np.array(b).mean())
#for i in range(len(a))
   # b.append(a[i, min(len(a)-1,  i+10)])
#plt.plot(b)

2.5


In [115]:
#  this is a test
#

class Test:
    a = 1
    b = 2
    aa = 1
    
    def __init__(self, c, d = 20, e=30):
        self.a = c
        self.b = d
        self.e = e
        print([self.a, self.b, self.e])
        
    #同名の関数は、後に定義されたもので上書きされるらしい。
    
    def test0(self):
        print(1)
    def test2(self):
        print(2 + self.a)
    def test3(self):
        self.test2()
        self.test0()

t  = Test(5)
u = Test(10,e=100)
t.test3()
print(t.a)

[5, 20, 30]
[10, 20, 100]
7
1
5


In [374]:
#  this is a test
#
pd.DataFrame([np.ones(300)*70,np.ones(300)* 25,np.ones(300)*5]).to_csv('rate_table/stable70_25_5.csv', index=False, header=None)

a = pd.read_csv('rate_table/' + 'rateTable_size20_300trials_001.csv', header=None)
b = pd.read_csv('rate_table/' + 'stable70_25_5.csv', header=None)
print(a)
print(b)

   0    1    2    3    4    5    6    7    8    9    ...  290  291  292  293  \
0    1    2    4    3    4    5    6    8    8    9  ...   32   33   33   34   
1   59   57   54   53   52   51   50   51   56   59  ...   51   49   47   44   
2   39   40   41   43   43   42   43   40   35   31  ...   16   17   18   20   

   294  295  296  297  298  299  
0   34   32   31   30   28   27  
1   45   47   46   47   48   50  
2   20   20   21   22   22   21  

[3 rows x 300 columns]
    0     1     2     3     4     5     6     7     8     9    ...   290  \
0  70.0  70.0  70.0  70.0  70.0  70.0  70.0  70.0  70.0  70.0  ...  70.0   
1  25.0  25.0  25.0  25.0  25.0  25.0  25.0  25.0  25.0  25.0  ...  25.0   
2   5.0   5.0   5.0   5.0   5.0   5.0   5.0   5.0   5.0   5.0  ...   5.0   

    291   292   293   294   295   296   297   298   299  
0  70.0  70.0  70.0  70.0  70.0  70.0  70.0  70.0  70.0  
1  25.0  25.0  25.0  25.0  25.0  25.0  25.0  25.0  25.0  
2   5.0   5.0   5.0   5.0   5.0   5.0   

In [104]:
import csv
import pprint
import numpy as np
import pandas as pd

with open('test2.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow([0, 1, 2])
    writer.writerow(['a', 'b', 'c',[[2,3],[4,5]]])
p = pd.read_csv('test2.csv')
print(np.array(eval(p['2'][0])))


[[2 3]
 [4 5]]


In [85]:
a =[1,2]
print(a)
a.append(3)
print(a)
a.append([4,5])
print(a)
a.extend([6,7])
print(a)
a.extend([])
print(a)

[1, 2]
[1, 2, 3]
[1, 2, 3, [4, 5]]
[1, 2, 3, [4, 5], 6, 7]
[1, 2, 3, [4, 5], 6, 7]


In [351]:
a =[]
a.append([1,2])
a.append([3,4])
a.append([5,6])
print(a)

[[1, 2], [3, 4], [5, 6]]


In [4]:
np.array([1,2]).dot([1,2])

5

In [41]:
# test

class Fish:
    def __init__(self, name, build="ほね", eyelids=False):
        self.name = name
        self.build = build 
        self.eyelids = eyelids

    def swim(self):
        print("こちらの魚は泳ぎます")

    def swim_back(self):
        print("こちらの魚は後ろ向きにも泳ぎます")

class Medaka(Fish):
    def __init__(self, name,a):
        super(Medaka,self).__init__(name)
        return
        
    def p(self):
        print(str(self.name)+ " " + str(self.build))
    

a = Medaka("namae",6)
a.p()

namae ほね


In [34]:
def get_select_rate(self, op):
    op = np.array(op)
    ex = math.exp(self.beta * op)
    below = sum(ex)/100.0
    return op/below

合計：150円合計：20円

In [60]:
a = np.array(["a", "b", "c"])
b = np.array(["A", "B", "C"])
for i in range(len(a)):
    print(a[i], end=" ")
print()
for ch in a:
    print(ch, end=" ")
print()
for ch, ch2 in zip(a, b):
    print("{}-{}".format(ch2,ch), end = " ")

a b c 
a b c 
A-a B-b C-c 

In [69]:
np.log(1e-5)

-11.512925464970229